In [24]:
from collections import defaultdict
from __future__ import annotations
from typing import List, Dict, Tuple
import random
import numpy as np

In [34]:
class Simplex:
    def __init__(self, vertices: List[int]):
        self.n = len(vertices)
        self.vertices = vertices

    def get_representation(self) -> List[int]:
        return sorted(self.vertices)

    def __eq__(self, other: Simplex) -> bool:
        return self.get_representation() == other.get_representation()

    def get_immediate_children(self) -> List[Simplex]:
        if self.n == 1:
            return []
        return [
            Simplex([*self.vertices[:i], *self.vertices[i + 1 :]])
            for i in range(self.n)
        ]

    def __str__(self) -> str:
        return f"Simplex({' '.join(str(v) for v in self.get_representation())})"

    def __repr__(self) -> str:
        return self.__str__()


a = Simplex([1, 2, 3])
print(a)
print(a.get_immediate_children())

Simplex(1 2 3)
[Simplex(2 3), Simplex(1 3), Simplex(1 2)]


In [42]:
class SimplicialComplex:
    def __init__(self, simplices: List[Simplex]):
        # It will be useful later to keep the simplices graded by dimension.
        self.graded_simplices = defaultdict(list)
        for simplex in simplices:
            self.graded_simplices[simplex.n].append(simplex)
        self.largest_n = max(self.graded_simplices.keys())
        self.fill_in_gaps()

    def fill_in_simplex(self, simplex: Simplex):
        if simplex in self.graded_simplices[simplex.n]:
            return
        self.graded_simplices[simplex.n].append(simplex)
        for child in simplex.get_immediate_children():
            self.fill_in_simplex(child)

    def fill_in_gaps(self):
        for n in range(1, self.largest_n + 1):
            for simplex in self.graded_simplices[n]:
                for child in simplex.get_immediate_children():
                    self.fill_in_simplex(child)

    def __str__(self) -> str:
        return f"SimplicialComplex({dict(self.graded_simplices)})"

    def __repr__(self) -> str:
        return self.__str__()


a, b = Simplex([1, 2, 3]), Simplex([2, 3])
X = SimplicialComplex([a, b])
print(X)

SimplicialComplex({3: [Simplex(1 2 3)], 2: [Simplex(2 3), Simplex(1 3), Simplex(1 2)], 1: [Simplex(3), Simplex(2), Simplex(1)]})


In [43]:
class CellularSheaf:
    def __init__(self, simplicial_complex: SimplicialComplex):
        """
        In this version of a cellular sheaf, we will assume that each simplex is assigned R,
        and that the boundary maps are real numbers.
        """

        self.simplicial_complex = simplicial_complex
        self.largest_n = simplicial_complex.largest_n

        # We check that the boundary condition holds
        self.check_valid_simplicial_complex()

    def check_valid_simplicial_complex(self):
        for n in range(1, self.largest_n + 1):
            for simplex in self.simplicial_complex.graded_simplices[n]:
                for child in simplex.get_immediate_children():
                    if child not in self.simplicial_complex.graded_simplices[n - 1]:
                        raise ValueError(
                            f"Simplex {child} is not in the simplicial complex"
                        )

    def generate_random_path_value_mapping(self) -> Dict[str, float]:
        path_value_map = {}
        for n in range(1, self.largest_n + 1):
            for simplex in self.simplicial_complex.graded_simplices[n]:
                # For now, set to be random number between 1 and 10
                # Since Simplex is a class we can't hash it, so we need to use the representation
                path_value_map[str(simplex)] = random.random() * 9 + 1
        return path_value_map

    def get_boundary_map_matrices(
        self, path_value_map: Dict[str, float]
    ) -> Dict[(int, int), List[List[float]]]:

        boundary_map_matrices = {}
        for a in range(1, self.largest_n):
            b = a + 1
            # Note that b is for the larger dimensional simplices
            input_dim, output_dim = (
                len(self.simplicial_complex.graded_simplices[b]),
                len(self.simplicial_complex.graded_simplices[a]),
            )
            boundary_map = np.zeros((input_dim, output_dim))
            for i in range(input_dim):
                current_simplex = self.simplicial_complex.graded_simplices[b][i]
                desired_path_value = path_value_map[str(current_simplex)]
                sign = 1
                for j in range(output_dim):
                    current_child = self.simplicial_complex.graded_simplices[a][j]
                    if current_child in current_simplex.get_immediate_children():
                        child_path_value = path_value_map[str(current_child)]
                        boundary_map[i, j] = sign * desired_path_value / child_path_value
                        sign *= -1
                    else:
                        boundary_map[i, j] = 0

            boundary_map_matrices[(b, a)] = boundary_map
        return boundary_map_matrices

print(X.graded_simplices)
C = CellularSheaf(X)
path_value_map = C.generate_random_path_value_mapping()
print(path_value_map)
boundary_map_matrices = C.get_boundary_map_matrices(path_value_map)
print(boundary_map_matrices)

print(boundary_map_matrices[(3, 2)] @ boundary_map_matrices[(2, 1)])

defaultdict(<class 'list'>, {3: [Simplex(1 2 3)], 2: [Simplex(2 3), Simplex(1 3), Simplex(1 2)], 1: [Simplex(3), Simplex(2), Simplex(1)]})
{'Simplex(3)': 8.852471270791602, 'Simplex(2)': 4.939319364837564, 'Simplex(1)': 6.152319305075488, 'Simplex(2 3)': 7.497281698101757, 'Simplex(1 3)': 6.306988342593684, 'Simplex(1 2)': 7.645061117095947, 'Simplex(1 2 3)': 7.8372757986998645}
{(2, 1): array([[ 0.84691398, -1.51787749,  0.        ],
       [ 0.7124551 ,  0.        , -1.02513996],
       [ 0.        ,  1.54779648, -1.24263074]]), (3, 2): array([[ 1.04534898, -1.24263363,  1.02514233]])}
[[-8.38277217e-17 -1.10460933e-16 -1.69258233e-16]]


TODO:
- Tests for the generated boundary maps
- Random cellular complex generation